NMFk: Extraction of signals from observation points (wells) detecting mixed concentration
---

Let us assume that there are 20 wells (monitoring points), 2 sources (contaminant plumes), and 3 chemical species (e.g., nitrate, sulfate, cloride) detected at each well.

These types of analyses are applicable for many cases in which mixing of signals (volumes/mass) is constrained by volumetric or weight constraints.
For example, these types of problmes occur in the case of mass transport in fluids (e.g., atmosphere, oceans, watersheds, aquifers, etc.).

In [1]:
import NMFk
import Random

┌ Info: Recompiling stale cache file /Users/monty/.julia/compiled/v1.2/NMFk/Ywuhu.ji for NMFk [e40cd9e2-a1df-5d90-a1fa-603fdc3dbdd8]
└ @ Base loading.jl:1240
┌ Warning: PyPlot is using tkagg backend, which is known to cause crashes on MacOS (#410); use the MPLBACKEND environment variable to request a different backend.
└ @ PyPlot ~/.julia/packages/PyPlot/4wzW1/src/init.jl:192
  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **



NMFk: Nonnegative Matrix Factorization + k-means clustering
====

  _     _   _      _   _______   _
 |  \  | | |  \  /  | |  _____| | |  _
 | . \ | | | . \/ . | | |___    | | / /
 | |\ \| | | |\  /| | |  ___|   | |/ /
 | | \ ' | | | \/ | | | |       |   (
 | |  \  | | |    | | | |       | |\ \
 |_|   \_| |_|    |_| |_|       |_| \_\

NMFk performs unsupervised machine learning based on matrix decomposition coupled with sparsity and nonnegativity constraints.
NMFk methodology allows for automatic identification of the optimal number of features (signals) present in two-dimensional data arrays (matrices).
The number of features is estimated automatically.


HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-6292203269370384457\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-17139355282912406372\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-6292203269370384457\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")

┌ Info: Loading DataFrames support into Gadfly.jl
└ @ Gadfly /Users/monty/.julia/packages/Gadfly/09PWZ/src/mapping.jl:228
┌ Warning: PyPlot is using tkagg backend, which is known to cause crashes on MacOS (#410); use the MPLBACKEND environment variable to request a different backend.
└ @ PyPlot /Users/monty/.julia/packages/PyPlot/4wzW1/src/init.jl:192


In [2]:
nWells = 20
nSources = 2
nSpecies = 3
Random.seed!(2015);

Due to volumetric constraints, mixing at each of well (observation pooint) has to add to 1.

The mixing matrix $W$ is defined as:

In [3]:
W = rand(nWells, nSources)
for i = 1:nWells
	W[i, :] ./= sum(W[i, :])
end
display(W)

20×2 Array{Float64,2}:
 0.395136  0.604864
 0.231592  0.768408
 0.351722  0.648278
 0.562318  0.437682
 0.476007  0.523993
 0.547533  0.452467
 0.446452  0.553548
 0.499894  0.500106
 0.601403  0.398597
 0.267675  0.732325
 0.592915  0.407085
 0.55854   0.44146 
 0.735353  0.264647
 0.364806  0.635194
 0.121244  0.878756
 0.788195  0.211805
 0.245554  0.754446
 0.818814  0.181186
 0.450388  0.549612
 0.507392  0.492608

$W$ defines 2 mixing values for each of the 20 observation points.

The 2 mixing values defines how the 2 sources (signals) are mixed at each well.

The mixing values for each well add up to 1.

Let us define $H$ as a matrix which specifies the concentrations of the 3 chemical species present in the 2 sources:

In [4]:
H = [100 0 3; 5 10 20]

2×3 Array{Int64,2}:
 100   0   3
   5  10  20

Here, the first source has elevated (100) concentration for the first species.

The second source has elevated (100) concentrations for the second (10) and third (20) species.

Now we can compute the observed concetrations at the wells $X$ as:

In [5]:
X = W * H

20×3 Array{Float64,2}:
 42.5379  6.04864  13.2827 
 27.0013  7.68408  16.0629 
 38.4136  6.48278  14.0207 
 58.4202  4.37682  10.4406 
 50.2206  5.23993  11.9079 
 57.0156  4.52467  10.6919 
 47.413   5.53548  12.4103 
 52.4899  5.00106  11.5018 
 62.1333  3.98597   9.77616
 30.4291  7.32325  15.4495 
 61.3269  4.07085   9.92044
 58.0613  4.4146   10.5048 
 74.8585  2.64647   7.49901
 39.6565  6.35194  13.7983 
 16.5182  8.78756  17.9389 
 79.8785  2.11805   6.60068
 28.3277  7.54446  15.8256 
 82.7873  1.81186   6.08016
 47.7869  5.49612  12.3434 
 53.2022  4.92608  11.3743 

We can also make some of the observations ''missing'' by setting their values to $NaN$:

In [7]:
X[1, 1] = NaN
display(X)

20×3 Array{Float64,2}:
 NaN       6.04864  13.2827 
  27.0013  7.68408  16.0629 
  38.4136  6.48278  14.0207 
  58.4202  4.37682  10.4406 
  50.2206  5.23993  11.9079 
  57.0156  4.52467  10.6919 
  47.413   5.53548  12.4103 
  52.4899  5.00106  11.5018 
  62.1333  3.98597   9.77616
  30.4291  7.32325  15.4495 
  61.3269  4.07085   9.92044
  58.0613  4.4146   10.5048 
  74.8585  2.64647   7.49901
  39.6565  6.35194  13.7983 
  16.5182  8.78756  17.9389 
  79.8785  2.11805   6.60068
  28.3277  7.54446  15.8256 
  82.7873  1.81186   6.08016
  47.7869  5.49612  12.3434 
  53.2022  4.92608  11.3743 

Now assiming that only $X$ is **known**, we can estimate **unknown** $W$ and $H$ using **NMFk**.

The **NMFk** provides estimates of $W$ and $H$ ($We$ and $He$) together with two variables evaluating accounts for the mixing.

In [10]:
We, He, fit, aic, kopt = NMFk.execute(X, 2:4; mixture=:mixmatch)

┌ Info: Saving requested but casefilename not specified; casefilename = "nmfk" will be used!
└ @ NMFk /Users/monty/.julia/dev/NMFk/src/NMFkExecute.jl:32



OF: min 2.3544459195637997e-17 max 1.1873067515517992e-16 mean 6.693127765578324e-17 std 2.986507206492013e-17
Worst correlation by columns: 24.25443447224253
Worst correlation by rows: 3.423472891487247
Worst norm by columns: 9.53474385418074e-12
Worst norm by rows: 1.9796606763520907e-11
Signals:  2 Fit:     22.28706 Silhouette:    0.9993618 AIC:    -5.438354


┌ Info: Saving requested but casefilename not specified; casefilename = "nmfk" will be used!
└ @ NMFk /Users/monty/.julia/dev/NMFk/src/NMFkExecute.jl:32



OF: min 1.2427222624536149e-16 max 3.113707918756757e-12 mean 3.1210148508796577e-13 std 9.84384926612985e-13
Worst correlation by columns: 24.254434473439872
Worst correlation by rows: 3.4234728900612303
Worst norm by columns: 2.3361147385761104e-11
Worst norm by rows: 3.73835270580918e-11
Signals:  3 Fit:     19.94667 Silhouette:   -0.3958711 AIC:     34.01596


┌ Info: Saving requested but casefilename not specified; casefilename = "nmfk" will be used!
└ @ NMFk /Users/monty/.julia/dev/NMFk/src/NMFkExecute.jl:32



OF: min 1.5453666579032498e-16 max 3.4463530169198547e-13 mean 4.77220417515629e-14 std 1.0692223519507712e-13
Worst correlation by columns: 24.254434467962795
Worst correlation by rows: 3.423472890220355
Worst norm by columns: 4.061325067072703e-11
Worst norm by rows: 4.889344533047362e-11
Signals:  4 Fit:     38.20362 Silhouette:   -0.5337855 AIC:     118.3582
Signals:  2 Fit:     22.28706 Silhouette:    0.9993618 AIC:    -5.438354
Signals:  3 Fit:     19.94667 Silhouette:   -0.3958711 AIC:     34.01596
Signals:  4 Fit:     38.20362 Silhouette:   -0.5337855 AIC:     118.3582


┌ Info: Results
└ @ NMFk /Users/monty/.julia/dev/NMFk/src/NMFkExecute.jl:19
┌ Info: Optimal solution: 2 signals
└ @ NMFk /Users/monty/.julia/dev/NMFk/src/NMFkExecute.jl:25


(Array{Float64,2}[#undef, [0.4283139894710423 0.5716860105289577; 0.2482348459648578 0.7517651540351423; … ; 0.4891521342526552 0.5108478657473448; 0.551919058816045 0.4480809411839551], [0.30355613035335766 0.2889476605953807 0.4074962090512617; 0.39485842265067794 0.377520193320631 0.22762138402869098; … ; 0.27271053882600255 0.259024133082174 0.46826532809182353; 0.24088697208957988 0.22815199044471846 0.5309610374657018], [0.2014424687690873 0.22977538816119045 0.19743315578487097 0.3713489872848513; 0.2809790709537467 0.2661710493400927 0.24017481746294828 0.21267506224321236; … ; 0.1910342934879941 0.18818744721956845 0.17121081409284328 0.44956744519959424; 0.1676007043754704 0.16787013851354432 0.15324340835535807 0.5112857487556273]], Array{Float64,2}[#undef, [92.58201488213643 0.7808405415550482 4.327428916232103; 6.997205223875943 9.789767868986862 19.642605380749213], [7.523238785602447 12.381519947011755 15.139112904889668; 5.144865895439195 7.517814076585946 24.0832959262